In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import tensorflow as tf
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Conv2D,Concatenate,Flatten,Add,Dropout,GRU
import random
import datetime
from nltk.translate.bleu_score import sentence_bleu
from math import log
import torch
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
import os
from PIL import Image
import numpy as np
import cv2
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dataset = pd.read_csv('/kaggle/input/data-split-csv/Train_Data.csv')
cv_dataset = pd.read_csv('/kaggle/input/data-split-csv/CV_Data.csv')
test_dataset = pd.read_csv('/kaggle/input/data-split-csv/Test_Data.csv')
X_train = train_dataset['Person_id']
y_train = train_dataset['Report']

X_cv = cv_dataset['Person_id']
y_cv = cv_dataset['Report']

X_test = test_dataset['Person_id']
y_test = test_dataset['Report']

max_capt_len = 153
pad_size = max_capt_len 

In [10]:
import os
import re
import cv2
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
warnings.filterwarnings('ignore')
import xml.etree.ElementTree as ET
from wordcloud import WordCloud, ImageColorGenerator
cwd = os.getcwd()
import tarfile
images = tarfile.open('/kaggle/input/images-reports/NLMCXR_png.tgz')
images.extractall(cwd+'/xray_images/')

In [4]:
train_dataset

,Person_id,Image1,Image2,Report
0,Scanned Images/CXR443_IM-2078_0,Scanned Images/CXR443_IM-2078-1001.png,Scanned Images/CXR443_IM-2078-1002.png,startseq heart size and mediastinal contour wi...
1,Scanned Images/CXR2552_IM-1058_0,Scanned Images/CXR2552_IM-1058-1001.png,Scanned Images/CXR2552_IM-1058-1002.png,startseq lungs are clear without focal consoli...
2,Scanned Images/CXR1622_IM-0404_0,Scanned Images/CXR1622_IM-0404-1001.png,Scanned Images/CXR1622_IM-0404-2001.png,startseq the cardiomediastinal silhouette with...
3,Scanned Images/CXR3986_IM-2041_0,Scanned Images/CXR3986_IM-2041-1001.png,Scanned Images/CXR3986_IM-2041-2001.png,startseq the cardiomediastinal silhouette and ...
4,Scanned Images/CXR3986_IM-2041_1,Scanned Images/CXR3986_IM-2041-1001.png,Scanned Images/CXR3986_IM-2041-3001.png,startseq the cardiomediastinal silhouette and ...
...,...,...,...,...
2745,Scanned Images/CXR3102_IM-1454_0,Scanned Images/CXR3102_IM-1454-1001.png,Scanned Images/CXR3102_IM-1454-2001.png,startseq the lungs are clear bilaterally . sp...
2746,Scanned Images/CXR1634_IM-0414_0,Scanned Images/CXR1634_IM-0414-2002.png,Scanned Images/CXR1634_IM-0414-3003.png,startseq calcified granulomas . calcified hil...
2747,Scanned Images/CXR3849_IM-1947_0,Scanned Images/CXR3849_IM-1947-1001.png,Scanned Images/CXR3849_IM-1947-2001.png,startseq cardiomediastinal silhouettes are wit...
2748,Scanned Images/CXR1606_IM-0394,Scanned Images/CXR1606_IM-0394-2001.png,Scanned Images/CXR1606_IM-0394-2001.png,startseq the heart normal size . the mediasti...


In [5]:
# Print first few reports to check the data
print("First 5 reports:")
for i in range(5):
    print(f"\nReport {i+1}:")
    print(train_dataset['Report'][i])
    # Split report into words and get unique word count
    words = set(str(train_dataset['Report'][i]).split())
    print(f"Unique words: {len(words)}")

# Calculate max vocabulary size per report
max_len = max([len(set(str(report).split())) for report in train_dataset['Report']])
print(f"\nMaximum vocabulary size in a single report: {max_len}")

First 5 reports:

Report 1:
startseq heart size and mediastinal contour within normal limits .  no focal airspace consolidation pneumothora large pleural effusion .  degenerative changes thoracic spine .  endseq
Unique words: 23

Report 2:
startseq lungs are clear without focal consolidation effusion pneumothora .  normal heart size .  bony thora and soft tissues grossly unremarkable endseq
Unique words: 21

Report 3:
startseq the cardiomediastinal silhouette within normal limits for size and contour .  the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothora .  no acute osseus abnormality .  endseq
Unique words: 29

Report 4:
startseq the cardiomediastinal silhouette and vasculature are within normal limits for size and contour .  the lungs are normally inflated and clear .  osseous structures are within normal limits for patient age .  endseq
Unique words: 23

Report 5:
startseq the cardiomediastinal silhouette and vasculature are within no

In [11]:
import os
import torch
import torch.nn as nn # Need nn for Sequential
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2

# --- Import ResNet and Transforms (already done in Cell 1) ---
import torchvision.models as models
import torchvision.transforms as transforms

# --- Setup Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Pre-trained ResNet-152 ---
print("Loading pre-trained ResNet-152...")
# Load the pre-trained model with weights appropriate for inference
# Using the updated API for weights is recommended
resnet_model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)

# Remove the final fully connected layer (classification layer)
# Features are extracted from the layer before it (avgpool)
# Access layers by name or structure (safer with names if possible, but structure works)
modules = list(resnet_model.children())[:-1] # Exclude the last layer (fc)
resnet_feature_extractor = nn.Sequential(*modules)

# Set the model to evaluation mode and move to device
resnet_feature_extractor.eval()
resnet_feature_extractor = resnet_feature_extractor.to(device)
print("ResNet-152 feature extractor loaded and moved to device.")

# --- Define ImageNet Preprocessing Transforms ---
# Standard transforms for ImageNet models
# Ensure input images are resized/cropped to 224x224
preprocess = transforms.Compose([
    transforms.Resize(256),             # Resize smaller side to 256
    transforms.CenterCrop(224),         # Crop center 224x224
    transforms.ToTensor(),              # Convert image to PyTorch Tensor (scales to [0, 1])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize
])

# --- Updated Image Loading and Feature Extraction Functions ---

def load_and_preprocess_image(img_path):
    """Loads an image, ensures 3 channels (RGB), and applies ResNet preprocessing."""
    try:
        img = Image.open(img_path).convert('RGB') # Ensure image is RGB
        img_tensor = preprocess(img)
        # Add batch dimension (unsqueeze(0)) -> Shape: [1, 3, 224, 224]
        return img_tensor.unsqueeze(0)
    except FileNotFoundError:
        print(f"⚠️ File not found: {img_path}")
        return None
    except Exception as e:
        print(f"⚠️ Error processing image {img_path}: {e}")
        return None

def extract_img_feature(image1_rel_path, image2_rel_path, base_path="/kaggle/working/xray_images"):
    """Extracts and concatenates ResNet-152 features for two images."""

    # Helper function to extract features for a single image path (relative path)
    def extract_single_feature(relative_img_path):
        # Construct full path using the base path and relative path
        filename = os.path.basename(relative_img_path.strip())
        full_path = os.path.join(base_path, filename)

        # Load and preprocess the image
        image_tensor = load_and_preprocess_image(full_path)
        if image_tensor is None:
            return None # Return None if image loading/preprocessing failed

        # Move tensor to the correct device (GPU or CPU)
        image_tensor = image_tensor.to(device)

        # Extract features using ResNet feature extractor
        with torch.no_grad(): # Disable gradient calculation for inference
            features = resnet_feature_extractor(image_tensor)
            # Output shape after nn.Sequential(*list(resnet_model.children())[:-1])
            # is typically [batch_size, num_features, 1, 1] (e.g., [1, 2048, 1, 1])
            # We need to flatten it to get a feature vector
            features_flat = torch.flatten(features, 1) # Shape: [1, 2048]

        # Squeeze unnecessary dimensions and move features to CPU as a NumPy array
        return features_flat.squeeze().cpu().numpy() # Return as NumPy array (shape: (2048,))

    # Extract features for both images using the helper function
    img1_features = extract_single_feature(image1_rel_path)
    img2_features = extract_single_feature(image2_rel_path)

    # Concatenate features if both were extracted successfully
    if img1_features is not None and img2_features is not None:
        # Concatenate the 1D NumPy arrays
        combined_feature = np.concatenate([img1_features, img2_features], axis=0)
        # Resulting shape: (4096,)
        return combined_feature
    else:
        # Handle cases where one or both images failed to process
        print(f"❌ Could not extract features for one or both images: {image1_rel_path}, {image2_rel_path}")
        # Return None to indicate failure for this pair
        return None

def extract_features_for_datasets(image_df, base_path):
    """Extracts features for all image pairs in a DataFrame, handles failures."""
    features_list = []
    successful_indices = [] # Keep track of original DataFrame indices

    # Iterate through DataFrame rows with a progress bar
    print(f"\nStarting feature extraction for {len(image_df)} pairs...")
    for index, row in tqdm(image_df.iterrows(), total=image_df.shape[0], desc="Extracting ResNet features"):
        # Get relative image paths from the DataFrame
        img1_rel_path = row['Image1']
        img2_rel_path = row['Image2']

        # Extract combined features for the image pair
        feature = extract_img_feature(img1_rel_path, img2_rel_path, base_path)

        # Only append features and index if extraction was successful
        if feature is not None:
            features_list.append(feature)
            successful_indices.append(index) # Store the original index

    # Convert the list of successful features into a single NumPy array
    features_array = np.array(features_list)
    print(f"\nSuccessfully extracted features for {len(successful_indices)} out of {len(image_df)} image pairs.")

    # Return the feature array and the list of successful original indices
    return features_array, successful_indices

# --- Prepare DataFrames (assuming train_dataset, cv_dataset, test_dataset loaded in Cell 2) ---
print("\nPreparing DataFrames...")
# Select necessary columns, keeping the original index
train_images_df = train_dataset[['Image1', 'Image2', 'Report']].copy()
cv_images_df = cv_dataset[['Image1', 'Image2', 'Report']].copy()
test_images_df = test_dataset[['Image1', 'Image2', 'Report']].copy()
print("DataFrames prepared.")

# --- Run Feature Extraction ---
# Define the base path where images were extracted (e.g., by tarfile in Cell 3)
base_image_path = '/kaggle/working/xray_images' # Verify this path is correct
print(f"Base image path set to: {base_image_path}")

# Extract features for training, validation, and test sets
X_train_features, train_indices = extract_features_for_datasets(train_images_df, base_image_path)
X_cv_features, cv_indices = extract_features_for_datasets(cv_images_df, base_image_path)
X_test_features, test_indices = extract_features_for_datasets(test_images_df, base_image_path)

# --- Filter Reports based on Successful Feature Extraction ---
# Ensure the reports correspond exactly to the successfully extracted features
# Use the stored original indices (.loc) to select the correct reports
print("\nFiltering reports to match successful feature extractions...")
y_train_filtered = train_images_df.loc[train_indices, 'Report'].tolist()
y_cv_filtered = cv_images_df.loc[cv_indices, 'Report'].tolist()
y_test_filtered = test_images_df.loc[test_indices, 'Report'].tolist()
print("Reports filtered.")

# --- Reassign variables for consistency with subsequent cells ---
# Overwrite old variables with the filtered data
X_train = X_train_features
X_cv = X_cv_features
X_test = X_test_features
# Overwrite report lists with the filtered versions
y_train = y_train_filtered
y_cv = y_cv_filtered
y_test = y_test_filtered

# Print shapes of the final feature arrays and the number of corresponding reports
print("\n--- Final Data Shapes After Filtering ---")
print(f"X_train shape: {X_train.shape}, y_train count: {len(y_train)}")
print(f"X_cv shape: {X_cv.shape},    y_cv count: {len(y_cv)}")
print(f"X_test shape: {X_test.shape},  y_test count: {len(y_test)}")

# Sanity check: Number of samples should match between features and reports
assert X_train.shape[0] == len(y_train), "Mismatch between X_train features and y_train reports!"
assert X_cv.shape[0] == len(y_cv), "Mismatch between X_cv features and y_cv reports!"
assert X_test.shape[0] == len(y_test), "Mismatch between X_test features and y_test reports!"
print("Data shapes verified.")

print("\nResNet-152 feature extraction complete.")

# --- Optional: Clear ResNet model from memory if needed ---
# Uncomment the following lines if GPU memory is tight before loading BioBART
# del resnet_model
# del resnet_feature_extractor
# if device == torch.device("cuda"):
#     torch.cuda.empty_cache()
# print("ResNet model cleared from memory (optional).")

Using device: cuda
Loading pre-trained ResNet-152...
ResNet-152 feature extractor loaded and moved to device.

Preparing DataFrames...
DataFrames prepared.
Base image path set to: /kaggle/working/xray_images

Starting feature extraction for 2750 pairs...


Extracting ResNet features: 100%|██████████| 2750/2750 [02:08<00:00, 21.47it/s]



Successfully extracted features for 2750 out of 2750 image pairs.

Starting feature extraction for 563 pairs...


Extracting ResNet features: 100%|██████████| 563/563 [00:25<00:00, 21.80it/s]



Successfully extracted features for 563 out of 563 image pairs.

Starting feature extraction for 394 pairs...


Extracting ResNet features: 100%|██████████| 394/394 [00:18<00:00, 21.56it/s]


Successfully extracted features for 394 out of 394 image pairs.

Filtering reports to match successful feature extractions...
Reports filtered.

--- Final Data Shapes After Filtering ---
X_train shape: (2750, 4096), y_train count: 2750
X_cv shape: (563, 4096),    y_cv count: 563
X_test shape: (394, 4096),  y_test count: 394
Data shapes verified.

ResNet-152 feature extraction complete.


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
# Removed GPT-2 imports, BART imports are in Cell 1
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from tqdm import tqdm # Keep tqdm if used here
import re # Keep re if used here

class MLP(nn.Module):
    """ MLP to map concatenated ResNet features to BioBART's hidden size."""
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        # Simple MLP: Linear -> ReLU -> Dropout -> Linear -> ReLU -> Dropout -> Linear -> LayerNorm
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(0.2) # Increased dropout slightly
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout2 = nn.Dropout(0.2) # Increased dropout slightly
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.layer_norm = nn.LayerNorm(output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.layer_norm(x)
        return x

# --- Load BioBART Tokenizer and Model (Using BioBART Base) ---

biobart_model_name = "GanjinZero/biobart-base" # Or GanjinZero/biobart-large
print(f"Loading BioBART tokenizer: {biobart_model_name}")
# Load tokenizer associated with the chosen BioBART model
biobart_tokenizer = BartTokenizer.from_pretrained(biobart_model_name)

print(f"Loading BioBART model: {biobart_model_name}")
# Load the pre-trained BioBART model for sequence generation
biobart_model = BartForConditionalGeneration.from_pretrained(biobart_model_name)

# --- Initialize MLP ---
# Get the hidden size (d_model) from the loaded BioBART model's configuration
mlp_output_dim = biobart_model.config.d_model
print(f"BioBART hidden size (MLP output dim): {mlp_output_dim}")

# *** IMPORTANT: Update MLP input dimension for concatenated ResNet-152 features ***
# ResNet-152 feature dimension = 2048
# Concatenated feature dimension = 2048 * 2 = 4096
mlp_input_dim = 4096
mlp_hidden_dim = 1024 # Hidden dimension for the MLP (can be tuned)
print(f"Initializing MLP: Input Dim={mlp_input_dim}, Hidden Dim={mlp_hidden_dim}, Output Dim={mlp_output_dim}")

mlp = MLP(input_dim=mlp_input_dim, hidden_dim=mlp_hidden_dim, output_dim=mlp_output_dim)

# --- Setup Device (Ensure device is defined, usually from previous cells) ---
if 'device' not in locals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device defined in this cell: {device}")
else:
    print(f"Using device defined previously: {device}")

# Move models to the selected device
mlp.to(device)
biobart_model.to(device)
print(f"MLP and BioBART models moved to device: {device}")

# --- Display Tokenizer Info (Optional but helpful) ---
print(f"\n--- Tokenizer Info ---")
print(f"BioBART Tokenizer Vocabulary Size: {biobart_tokenizer.vocab_size}")
print(f"Pad Token: '{biobart_tokenizer.pad_token}' (ID: {biobart_tokenizer.pad_token_id})")
print(f"BOS Token: '{biobart_tokenizer.bos_token}' (ID: {biobart_tokenizer.bos_token_id})")
print(f"EOS Token: '{biobart_tokenizer.eos_token}' (ID: {biobart_tokenizer.eos_token_id})")
# Check the decoder start token ID used by the model configuration
print(f"Decoder Start Token ID: {biobart_model.config.decoder_start_token_id} (Should match EOS ID for standard BART)")

print("\nModel and MLP setup complete.")


Loading BioBART tokenizer: GanjinZero/biobart-base
Loading BioBART model: GanjinZero/biobart-base
BioBART hidden size (MLP output dim): 768
Initializing MLP: Input Dim=4096, Hidden Dim=1024, Output Dim=768
Using device defined previously: cuda
MLP and BioBART models moved to device: cuda

--- Tokenizer Info ---
BioBART Tokenizer Vocabulary Size: 50265
Pad Token: '<pad>' (ID: 1)
BOS Token: '<s>' (ID: 0)
EOS Token: '</s>' (ID: 2)
Decoder Start Token ID: 2 (Should match EOS ID for standard BART)

Model and MLP setup complete.


In [14]:
y_train[:10]

['startseq heart size and mediastinal contour within normal limits .  no focal airspace consolidation pneumothora large pleural effusion .  degenerative changes thoracic spine .  endseq',
 'startseq lungs are clear without focal consolidation effusion pneumothora .  normal heart size .  bony thora and soft tissues grossly unremarkable endseq',
 'startseq the cardiomediastinal silhouette within normal limits for size and contour .  the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothora .  no acute osseus abnormality .  endseq',
 'startseq the cardiomediastinal silhouette and vasculature are within normal limits for size and contour .  the lungs are normally inflated and clear .  osseous structures are within normal limits for patient age .  endseq',
 'startseq the cardiomediastinal silhouette and vasculature are within normal limits for size and contour .  the lungs are normally inflated and clear .  osseous structures are within normal limi

In [16]:
# Code from Cell 9 in the notebook file (should be Cell index 8)
from torch.utils.data import TensorDataset, DataLoader
import re
import numpy as np # Ensure numpy is imported

# Access mlp_input_dim defined in the previous cell (Cell 7)
if 'mlp_input_dim' not in locals():
     mlp_input_dim = 4096
     print(f"Warning: mlp_input_dim not found in locals. Defaulting to {mlp_input_dim}.")

# Clean text function (remains the same)
def clean_text(text):
    # ... (implementation) ...
    text = str(text); text = re.sub(r'[^\w\s.]', '', text); text = ' '.join(text.split()); return text

# Clean and preprocess the reports
print("Cleaning reports...")
try:
    # Ensure y_train, y_cv, y_test are the filtered lists from Cell 6
    if not isinstance(y_train, list) or not isinstance(y_cv, list) or not isinstance(y_test, list):
         raise TypeError("y_train, y_cv, y_test should be lists (filtered reports from Cell 6).")
    y_train_cleaned = [clean_text(report) for report in y_train]
    y_cv_cleaned = [clean_text(report) for report in y_cv]
    y_test_cleaned = [clean_text(report) for report in y_test]
    print(f"Cleaned {len(y_train_cleaned)} train, {len(y_cv_cleaned)} CV, {len(y_test_cleaned)} test reports.")
except Exception as e:
    print(f"Error during report cleaning: {e}")
    y_train_cleaned, y_cv_cleaned, y_test_cleaned = [], [], []

# Print sample reports after cleaning
# ... (prints samples) ...

# Normalize image features
print("\nNormalizing image features...")
def normalize_features(features_array, name):
    # ... (implementation uses axis=0 mean/std, handles zero std) ...
    if isinstance(features_array, np.ndarray) and features_array.size > 0:
        mean, std = features_array.mean(axis=0), features_array.std(axis=0)
        std = np.where(std < 1e-6, 1.0, std)
        print(f"Normalizing {name} features (shape: {features_array.shape})...")
        return (features_array - mean) / std
    else:
        print(f"Warning: {name} features not found/empty. Skipping normalization.")
        return np.empty((0, mlp_input_dim))

try:
    X_train_norm = normalize_features(X_train, 'X_train')
    X_cv_norm = normalize_features(X_cv, 'X_cv')
    X_test_norm = normalize_features(X_test, 'X_test')
except NameError:
     print("Error: X_train, X_cv, or X_test features not found (expected from Cell 6).")
     X_train_norm, X_cv_norm, X_test_norm = [np.empty((0, mlp_input_dim))]*3

# Print statistics of normalized features
# ... (prints stats) ...

# Convert features to tensors
print("\nConverting features to tensors...")
def features_to_tensor(features_array, name):
    # ... (implementation uses torch.tensor, .copy(), dtype=torch.float) ...
     if isinstance(features_array, np.ndarray) and features_array.size > 0:
         return torch.tensor(features_array.copy(), dtype=torch.float)
     else:
         print(f"Warning: {name} features empty/invalid. Creating empty tensor.")
         return torch.empty((0, mlp_input_dim), dtype=torch.float)

X_train_tensor = features_to_tensor(X_train_norm, 'X_train_norm')
X_cv_tensor = features_to_tensor(X_cv_norm, 'X_cv_norm')
X_test_tensor = features_to_tensor(X_test_norm, 'X_test_norm')

# --- Tokenize reports using BioBART tokenizer ---
print("\nTokenizing reports with BioBART tokenizer...")
max_length = 150 # Sequence length for reports
try:
    # ... (implementation uses biobart_tokenizer, handles empty lists) ...
    y_train_tensor = biobart_tokenizer(y_train_cleaned, add_special_tokens=True, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')["input_ids"] if y_train_cleaned else torch.empty((0, max_length)).long()
    y_cv_tensor = biobart_tokenizer(y_cv_cleaned, add_special_tokens=True, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')["input_ids"] if y_cv_cleaned else torch.empty((0, max_length)).long()
    y_test_tensor = biobart_tokenizer(y_test_cleaned, add_special_tokens=True, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')["input_ids"] if y_test_cleaned else torch.empty((0, max_length)).long()
    print("Tokenization complete.")
except NameError:
    print("Error: biobart_tokenizer not defined. Run Cell 7 first.")
    y_train_tensor, y_cv_tensor, y_test_tensor = [torch.empty((0, max_length)).long()]*3

# --- Create datasets and dataloaders ---
print("\nCreating Datasets and DataLoaders...")
batch_size = 16
def create_dataloader(X_tensor, y_tensor, batch_size, shuffle=False, name=""):
    # ... (implementation handles empty tensors, checks shape match) ...
    if X_tensor.shape[0] > 0 and y_tensor.shape[0] > 0:
        if X_tensor.shape[0] == y_tensor.shape[0]:
            dataset = TensorDataset(X_tensor, y_tensor)
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
            print(f"  {name} DataLoader created with {len(dataset)} samples.")
            return dataset, dataloader
        else:
            print(f"Error creating {name} DataLoader: Mismatch samples features ({X_tensor.shape[0]}) vs reports ({y_tensor.shape[0]}).")
            return None, None
    else:
        print(f"Warning: {name} data tensors empty. Cannot create DataLoader.")
        return None, None

train_dataset, train_loader = create_dataloader(X_train_tensor, y_train_tensor, batch_size, shuffle=True, name="Training")
cv_dataset, cv_loader = create_dataloader(X_cv_tensor, y_cv_tensor, batch_size, shuffle=False, name="Validation")
test_dataset, test_loader = create_dataloader(X_test_tensor, y_test_tensor, batch_size, shuffle=False, name="Test")

# Print dataset information summary
# ... (prints sizes) ...
# Print sample tokenization check
# ... (prints sample) ...
# Check tensor shapes
# ... (prints shapes) ...
# Check first batch from dataloader
# ... (prints batch shapes) ...

print("\nPreprocessing complete.")


Cleaning reports...
Cleaned 2750 train, 563 CV, 394 test reports.

Normalizing image features...
Normalizing X_train features (shape: (2750, 4096))...
Normalizing X_cv features (shape: (563, 4096))...
Normalizing X_test features (shape: (394, 4096))...

Converting features to tensors...

Tokenizing reports with BioBART tokenizer...
Tokenization complete.

Creating Datasets and DataLoaders...
  Training DataLoader created with 2750 samples.
  Validation DataLoader created with 563 samples.
  Test DataLoader created with 394 samples.

Preprocessing complete.


In [17]:
# Code from Cell 11 in the notebook file (should be Cell index 10)
from tqdm import tqdm
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os # Import os for path checking

# --- Redefine train_step and validate just in case they weren't run ---
# (Code for train_step and validate functions is assumed to be correct from previous step)
def train_step(mlp, biobart_model, inputs, labels, optimizer, device):
    # ... (Implementation: models.train(), move data, zero_grad, MLP forward, prepare decoder embeds, BART forward, loss.backward, clip_grad, optimizer.step) ...
    mlp.train(); biobart_model.train()
    inputs = inputs.to(device); labels = labels.to(device)
    optimizer.zero_grad()
    embedded_input = mlp(inputs)
    batch_size, seq_len = labels.size()
    decoder_inputs_embeds = embedded_input.unsqueeze(1).expand(-1, seq_len, -1)
    decoder_attention_mask = (labels != biobart_tokenizer.pad_token_id).long().to(device)
    outputs = biobart_model(inputs_embeds=decoder_inputs_embeds, labels=labels, decoder_attention_mask=decoder_attention_mask)
    loss = outputs.loss
    loss.backward()
    torch.nn.utils.clip_grad_norm_(list(mlp.parameters()) + list(biobart_model.parameters()), max_norm=1.0)
    optimizer.step()
    return loss.item()

def validate(mlp, biobart_model, val_loader, device):
    # ... (Implementation: models.eval(), no_grad, loop through loader, MLP forward, prepare decoder embeds, BART forward, accumulate loss) ...
    mlp.eval(); biobart_model.eval()
    total_val_loss = 0; num_batches = 0
    with torch.no_grad():
        for input_batch, label_batch in val_loader:
            input_batch = input_batch.to(device); label_batch = label_batch.to(device)
            embedded_input = mlp(input_batch)
            batch_size, seq_len = label_batch.size()
            decoder_inputs_embeds = embedded_input.unsqueeze(1).expand(-1, seq_len, -1)
            decoder_attention_mask = (label_batch != biobart_tokenizer.pad_token_id).long().to(device)
            outputs = biobart_model(inputs_embeds=decoder_inputs_embeds, labels=label_batch, decoder_attention_mask=decoder_attention_mask)
            total_val_loss += outputs.loss.item(); num_batches += 1
    return total_val_loss / num_batches if num_batches > 0 else float('inf')

# --- Training Setup ---
# Initialize optimizer
lr_mlp = 1e-4
lr_bart = 5e-5 # Fine-tuning LR for BioBART
optimizer = optim.AdamW([
    {'params': mlp.parameters(), 'lr': lr_mlp},
    {'params': biobart_model.parameters(), 'lr': lr_bart}
], weight_decay=0.01)

# Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# Training loop parameters
epochs = 10
best_val_loss = float('inf')
patience = 5
patience_counter = 0
# *** FIX: Update checkpoint path name ***
checkpoint_path = 'resnet152_mlp_biobart_best.pt' # Changed from vit_...

# Check if dataloaders exist
if 'train_loader' not in locals() or train_loader is None or \
   'cv_loader' not in locals() or cv_loader is None:
    print("Error: Training or validation dataloader is missing. Cannot start training.")
else:
    print(f"Starting training loop... Checkpoint path: {checkpoint_path}")
    if device == torch.device("cuda"): torch.cuda.empty_cache()

    for epoch in range(epochs):
        total_train_loss = 0; num_train_batches = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)

        # --- Training Phase ---
        mlp.train(); biobart_model.train()
        for batch_idx, (input_batch, label_batch) in enumerate(progress_bar):
            try:
                loss = train_step(mlp, biobart_model, input_batch, label_batch, optimizer, device)
                total_train_loss += loss; num_train_batches += 1
                progress_bar.set_postfix({'train_loss': f'{loss:.4f}'})

                # --- Generate Sample Text Periodically ---
                if batch_idx > 0 and batch_idx % 200 == 0: # Adjust frequency if needed
                    # ... (Implementation uses models.eval(), generate with beam search, decode, print) ...
                    mlp.eval(); biobart_model.eval()
                    with torch.no_grad():
                        sample_input = input_batch[:1].to(device)
                        sample_embed = mlp(sample_input)
                        sample_embed_gen = sample_embed.unsqueeze(1)
                        gen_ids = biobart_model.generate(
                            inputs_embeds=sample_embed_gen, max_length=100, num_beams=4,
                            early_stopping=True, pad_token_id=biobart_tokenizer.pad_token_id,
                            eos_token_id=biobart_tokenizer.eos_token_id
                        )
                        generated_text = biobart_tokenizer.decode(gen_ids[0], skip_special_tokens=True)
                        print(f"\n--- Sample generation (Epoch {epoch+1}, Batch {batch_idx}) ---")
                        print(f"{generated_text.strip()}")
                        print("----------------------------------------------------------\n")
                    mlp.train(); biobart_model.train() # Return to train mode

                # Clear cache periodically
                if batch_idx > 0 and batch_idx % 50 == 0 and device == torch.device("cuda"):
                    torch.cuda.empty_cache()

            except RuntimeError as e:
                # ... (OOM handling) ...
                if "out of memory" in str(e):
                    print(f"\nWarning: CUDA OOM Error in batch {batch_idx}. Skipping."); torch.cuda.empty_cache(); optimizer.zero_grad(); continue
                else: print(f"Runtime Error in batch {batch_idx}: {str(e)}"); raise e

        # --- Validation Phase ---
        avg_train_loss = total_train_loss / num_train_batches if num_train_batches > 0 else float('inf')
        avg_val_loss = validate(mlp, biobart_model, cv_loader, device)
        progress_bar.close()
        print(f"\nEpoch {epoch + 1}/{epochs} Summary:")
        print(f"  Average Training Loss: {avg_train_loss:.4f}")
        print(f"  Validation Loss:       {avg_val_loss:.4f}")

        # --- Learning Rate Scheduling & Checkpointing ---
        scheduler.step(avg_val_loss)
        if avg_val_loss < best_val_loss:
            print(f"  Validation loss improved ({best_val_loss:.4f} --> {avg_val_loss:.4f}). Saving model...")
            best_val_loss = avg_val_loss; patience_counter = 0
            # Save checkpoint
            torch.save({
                'epoch': epoch, 'mlp_state_dict': mlp.state_dict(), 'biobart_state_dict': biobart_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(), 'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss, 'val_loss': avg_val_loss,
                'biobart_model_name': biobart_model_name # Keep model name
            }, checkpoint_path) # Use updated path name
            print(f"  Model saved to {checkpoint_path}")
        else:
            patience_counter += 1
            print(f"  Validation loss did not improve. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch + 1} epochs.")
                break

    print("\nTraining finished.")

Starting training loop... Checkpoint path: resnet152_mlp_biobart_best.pt



Epoch 1/10 Summary:
  Average Training Loss: 0.7607
  Validation Loss:       0.4675
  Validation loss improved (inf --> 0.4675). Saving model...
  Model saved to resnet152_mlp_biobart_best.pt



Epoch 2/10 Summary:
  Average Training Loss: 0.4131
  Validation Loss:       0.4200
  Validation loss improved (0.4675 --> 0.4200). Saving model...
  Model saved to resnet152_mlp_biobart_best.pt



Epoch 3/10 Summary:
  Average Training Loss: 0.3568
  Validation Loss:       0.3980
  Validation loss improved (0.4200 --> 0.3980). Saving model...
  Model saved to resnet152_mlp_biobart_best.pt



Epoch 4/10 Summary:
  Average Training Loss: 0.3188
  Validation Loss:       0.3901
  Validation loss improved (0.3980 --> 0.3901). Saving model...
  Model saved to resnet152_mlp_biobart_best.pt


...


Epoch 6/10 Summary:
  Average Training Loss: 0.2638
  Validation Loss:       0.3882
  Validation loss did not improve. Patience: 1/5



Epoch 7/10 Summary:
  Average Training Loss: 0.2429
  Validation Loss:       0.3917
  Validation loss did not improve. Patience: 2/5



Epoch 8/10 Summary:
  Average Training Loss: 0.2231
  Validation Loss:       0.3966
  Validation loss did not improve. Patience: 3/5



Epoch 9/10 Summary:
  Average Training Loss: 0.1945
  Validation Loss:       0.4076
  Validation loss did not improve. Patience: 4/5



Epoch 10/10 Summary:
  Average Training Loss: 0.1820
  Validation Loss:       0.4090
  Validation loss did not improve. Patience: 5/5

Early stopping triggered after 10 epochs.

Training finished.


In [19]:
# Evaluation function using BioBART
import torch
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import numpy as np

# Ensure NLTK resources are downloaded
import nltk
# Download required NLTK resources
print("Downloading NLTK punkt tokenizer...")
nltk.download('punkt', quiet=True)
print("Downloading NLTK wordnet...")
nltk.download('wordnet', quiet=True)

def evaluate_model(mlp, biobart_model, test_loader, biobart_tokenizer, device):
    """Evaluates the model on the test set using BLEU, METEOR, and ROUGE metrics."""
    
    # Ensure models are in evaluation mode
    mlp.eval()
    biobart_model.eval()
    
    generated_texts = []
    reference_texts = []
    
    print("\nStarting evaluation on the test set...")
    # Disable gradient calculations
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc='Evaluating', leave=False)
        for input_batch, label_batch in progress_bar:
            # Move data to the evaluation device
            input_batch = input_batch.to(device)
            # label_batch remains on CPU for decoding references easily
            
            # --- Generate Predictions --- 
            # 1. Get image embedding from MLP
            embedded_input = mlp(input_batch) # Shape: (batch_size, hidden_size)
            # Prepare for generate: needs shape (batch_size, seq_len, hidden_size)
            embedded_input_gen = embedded_input.unsqueeze(1) # Shape: (batch_size, 1, hidden_size)
            
            try:
                # 2. Generate text using BioBART
                outputs = biobart_model.generate(
                    inputs_embeds=embedded_input_gen,
                    max_length=153, # Max length for generated report (adjust if needed)
                    num_beams=4,
                    early_stopping=True,
                    pad_token_id=biobart_tokenizer.pad_token_id,
                    eos_token_id=biobart_tokenizer.eos_token_id,
                    # Use model's default decoder_start_token_id
                )
                
                # --- Decode and Store --- 
                # Decode generated sequences
                batch_generated_texts = biobart_tokenizer.batch_decode(outputs, skip_special_tokens=True)
                # Decode reference sequences (original labels)
                batch_reference_texts = biobart_tokenizer.batch_decode(label_batch, skip_special_tokens=True)
                
                # Clean up any extra spaces and store
                for gen_text, ref_text in zip(batch_generated_texts, batch_reference_texts):
                    generated_texts.append(gen_text.strip())
                    reference_texts.append(ref_text.strip())
            
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"\nCUDA OOM during evaluation. Skipping batch.")
                    torch.cuda.empty_cache()
                    continue
                else:
                    print(f"Runtime Error during evaluation: {str(e)}")
                    raise e # Re-raise other errors
            
            # Clear GPU memory periodically if needed
            if device == torch.device("cuda") and progress_bar.n % 20 == 0: # Every 20 batches
                 torch.cuda.empty_cache()
        
        progress_bar.close()

    # --- Calculate Metrics --- 
    if not generated_texts or not reference_texts:
        print("Evaluation failed: No texts were generated or references found.")
        return
        
    # 1. BLEU Scores
    print("\nCalculating BLEU scores...")
    # Tokenize texts for BLEU calculation
    generated_tokens = [word_tokenize(text.lower()) for text in generated_texts]
    # Reference tokens need to be a list of lists for corpus_bleu
    reference_tokens = [[word_tokenize(text.lower())] for text in reference_texts]
    
    # Use smoothing function for robustness, especially for shorter texts or smaller test sets
    smoothing = SmoothingFunction().method1 
    bleu1 = corpus_bleu(reference_tokens, generated_tokens, weights=(1.0, 0, 0, 0), smoothing_function=smoothing)
    bleu2 = corpus_bleu(reference_tokens, generated_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
    bleu3 = corpus_bleu(reference_tokens, generated_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    bleu4 = corpus_bleu(reference_tokens, generated_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
    
    print(f"  BLEU-1: {bleu1:.4f}")
    print(f"  BLEU-2: {bleu2:.4f}") 
    print(f"  BLEU-3: {bleu3:.4f}")
    print(f"  BLEU-4: {bleu4:.4f}")
    
    # 2. METEOR Score
    print("\nCalculating METEOR score...")
    # NLTK's meteor_score expects tokenized references and hypotheses
    meteor_scores = [meteor_score([ref[0]], gen) for ref, gen in zip(reference_tokens, generated_tokens)]
    avg_meteor = np.mean(meteor_scores) # Use numpy for mean calculation
    print(f"  METEOR: {avg_meteor:.4f}")
    
    # 3. ROUGE Scores
    print("\nCalculating ROUGE scores...")
    rouge1_scores = []
    rouge2_scores = []
    rouge3_scores = []
    rouge4_scores = []
    rougeL_scores = []
    
    for ref, gen in zip(reference_tokens, generated_tokens):
        # Calculate ROUGE-1 (unigram overlap)
        rouge1_scores.append(calculate_rouge_n(ref[0], gen, n=1))
        
        # Calculate ROUGE-2 (bigram overlap)
        rouge2_scores.append(calculate_rouge_n(ref[0], gen, n=2))
        
        # Calculate ROUGE-3 (trigram overlap)
        rouge3_scores.append(calculate_rouge_n(ref[0], gen, n=3))
        
        # Calculate ROUGE-4 (4-gram overlap)
        rouge4_scores.append(calculate_rouge_n(ref[0], gen, n=4))
        
        # Calculate ROUGE-L (longest common subsequence)
        rougeL_scores.append(calculate_rouge_l(ref[0], gen))
    
    # Calculate averages
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rouge3 = np.mean(rouge3_scores)
    avg_rouge4 = np.mean(rouge4_scores)
    avg_rougeL = np.mean(rougeL_scores)
    
    # Print ROUGE scores
    print(f"  ROUGE-1: {avg_rouge1:.4f}")
    print(f"  ROUGE-2: {avg_rouge2:.4f}")
    print(f"  ROUGE-3: {avg_rouge3:.4f}")
    print(f"  ROUGE-4: {avg_rouge4:.4f}")
    print(f"  ROUGE-L: {avg_rougeL:.4f}")
    
    # --- Print Example Generations --- 
    print("\nExample Generations (Test Set):")
    num_examples = min(5, len(generated_texts)) # Show up to 5 examples
    for i in range(num_examples):
        print(f"\n--- Example {i+1} ---")
        print(f"  Generated: {generated_texts[i]}")
        print(f"  Reference: {reference_texts[i]}")
        print("-----------------")

# Helper function for ROUGE-N calculation
def calculate_rouge_n(ref_tokens, gen_tokens, n=1):
    """Calculates ROUGE-N F1 score based on n-gram overlap."""
    if len(ref_tokens) == 0 or len(gen_tokens) == 0:
        return 0.0
    
    # Create n-grams
    def get_ngrams(tokens, n):
        return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    
    ref_ngrams = get_ngrams(ref_tokens, n)
    gen_ngrams = get_ngrams(gen_tokens, n)
    
    if not ref_ngrams or not gen_ngrams:
        return 0.0
    
    # Count matching n-grams
    ref_ngram_counts = {}
    for ngram in ref_ngrams:
        ref_ngram_counts[ngram] = ref_ngram_counts.get(ngram, 0) + 1
    
    gen_ngram_counts = {}
    for ngram in gen_ngrams:
        gen_ngram_counts[ngram] = gen_ngram_counts.get(ngram, 0) + 1
    
    # Count overlapping n-grams
    overlap_count = 0
    for ngram, count in gen_ngram_counts.items():
        overlap_count += min(count, ref_ngram_counts.get(ngram, 0))
    
    # Calculate precision and recall
    precision = overlap_count / len(gen_ngrams) if gen_ngrams else 0.0
    recall = overlap_count / len(ref_ngrams) if ref_ngrams else 0.0
    
    # Calculate F1 score
    if precision + recall == 0:
        return 0.0
    f1 = (2 * precision * recall) / (precision + recall)
    return f1
        
# Helper function for ROUGE-L calculation using Longest Common Subsequence (LCS)
def calculate_rouge_l(ref_tokens, gen_tokens):
    """Calculates ROUGE-L F1 score based on LCS."""
    # Find the length of the Longest Common Subsequence
    m, n = len(ref_tokens), len(gen_tokens)
    if m == 0 or n == 0: return 0.0
    # Initialize DP table (LCS lengths)
    L = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if ref_tokens[i-1] == gen_tokens[j-1]:
                L[i][j] = L[i-1][j-1] + 1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
    lcs_len = L[m][n]
    
    # Calculate Precision, Recall, and F1 for ROUGE-L
    recall = lcs_len / m if m > 0 else 0.0
    precision = lcs_len / n if n > 0 else 0.0
    
    if recall + precision == 0:
        f1 = 0.0
    else:
        f1 = (2 * recall * precision) / (recall + precision)
    return f1

# --- Run Evaluation --- 
# Ensure the test_loader is defined and models are loaded (potentially best checkpoint)
if 'test_loader' in locals() and test_loader is not None:
    # Optional: Load the best model checkpoint before evaluating
    if os.path.exists(checkpoint_path):
        print(f"\nLoading best model from {checkpoint_path} for final evaluation...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        mlp.load_state_dict(checkpoint['mlp_state_dict'])
        biobart_model.load_state_dict(checkpoint['biobart_state_dict'])
        print(f"Best model (Epoch {checkpoint['epoch']+1}) loaded.")
    else:
        print(f"Warning: Checkpoint {checkpoint_path} not found. Evaluating with current model state.")
        
    evaluate_model(mlp, biobart_model, test_loader, biobart_tokenizer, device)
else:
    print("\nError: Test loader not defined. Cannot run evaluation.")



Loading best model from resnet152_mlp_biobart_best.pt for final evaluation...
Best model (Epoch 5) loaded.

Starting evaluation on the test set...



Calculating BLEU scores...
  BLEU-1: 0.3356
  BLEU-2: 0.2033
  BLEU-3: 0.1231
  BLEU-4: 0.0772

Calculating METEOR score...
  METEOR: 0.3145

Calculating ROUGE scores...
  ROUGE-1: 0.4431
  ROUGE-2: 0.1626
  ROUGE-3: 0.0576
  ROUGE-4: 0.0270
  ROUGE-L: 0.3623

Example Generations (Test Set):

--- Example 1 ---
  Generated: startseq the cardiomediastinal silhouette normal size and contour . no focal consolidation pneumothora large pleural effusion . negative for acute bone abnormality . endseq
  Reference: startseq there calcified granuloma left midlung . there round density within the anterior segment the right upper lobe . there are prominent interstitial opacities which may represent changes associated with fibrosis . heart size normal . no pneumothora . anterior segment upper lobe rounded focal density . could lung nodule . endseq
-----------------

--- Example 2 ---
  Generated: startseq the cardiomediastinal silhouette normal size and contour . no focal consolidation pneumothora 